# How to chat with ChatGPT

ChatGPT is powered by `gpt-3.5-turbo` and `gpt-4`, OpenAI's most advanced models.

PSOpenAI PowerShell module provides `Request-ChatGPT` command for using the OpenAI API throw PowerShell friendly style.

`Request-ChatGPT` take a series of messages as input, and return an AI-written message as output.

This guide illustrates how to use `Request-ChatGPT` with various parameters.

## 1. Import the PSOpenAI module and set API key

If you set the API key to the environment variable named `OPENAI_API_KEY`. PSOpenAI commands will use it implicitly.

In [ ]:
# imports
Import-Module ..\PSOpenAI.psd1

# Set OpenAI API Key
$env:OPENAI_API_KEY = '<Put your API key here>'

## 2. Ask one question to ChatGPT, and get an answer

`Request-ChatGPT` has some basic optional parameters:

+ `-Message`: The messages to ChatGPT.
+ `-Model`: The name of the model you want to use (e.g.: `gpt-3.5-turbo`, `gpt-4`). This is optional. If not specified, `gpt-3.5-turbo` will be used.

In [ ]:
$Response = Request-ChatGPT -Message "Hi, please tell me your name." -Model "gpt-3.5-turbo"
Write-Output $Response


id      : chatcmpl-75fhjNRzdwwLz4c7DILRiGDN89Z65
object  : chat.completion
model   : gpt-3.5-turbo-0301
usage   : @{prompt_tokens=16; completion_tokens=23; total_tokens=39}
choices : {@{message=; finish_reason=stop; index=0}}
created : 2023/04/16 4:18:27
Message : Hi, please tell me your name.
Answer  : {I am an AI language model developed by OpenAI, and my creators refer to me as GPT-3.}
History : {System.Collections.Specialized.OrderedDictionary, System.Collections.Specialized.Ordered
          Dictionary}




As you can see, the response object has a few fields:  

+ `id`: The ID of the request
+ `object`: The type of object returned
+ `model`: The full name of the model used to generate the response
+ `usage`: The number of tokens used to
+ `choices`: a list of completion objects
+ `created`: The timestamp of the request
+ `Message`: The message to the ChatGPT
+ `Answer`: a list of the response messages from ChatGPT
+ `History`: a list of message history of past dialogues


Extract just the reply with:

In [ ]:
$Response.Answer

I am an AI language model developed by OpenAI, and my creators refer to me as GPT-3.


## 3. Multiple messages with context preserved (conversation)

`Request-ChatGPT` accepts past dialogs from pipeline. Additional questions can be asked while maintaining context.

In [ ]:
$Response1 = Request-ChatGPT -Message "What is the population of the United States? Please answer briefly."
"[1]: " + $Response1.Answer

$Response2 = $Response1 | Request-ChatGPT -Message "Translate the previous answer into Japanese."
"[2]: " + $Response2.Answer

[1]: As of 2021, the population of the United States is estimated to be approximately 331 million people.
[2]: 2021年現在、アメリカ合衆国の人口は約3億3100万人と推定されています。


## 4. System messages (Role prompt)

The system message can be used to prime the assistant with different personalities or behaviors.

`Request-ChatGPT` has `-RolePrompt` parameter for specifiyng system message.

Be aware that the how much pay attention to the system message is depending on the model.

In [ ]:
$Response = Request-ChatGPT `
    -Message "What is the population of the United States? Please answer briefly." `
    -RolePrompt "Please answer in Russian."

$Response.Answer

Население Соединенных Штатов - около 329,5 миллионов человек.


Due to the effect of the system messages, questions asked in English were answered in Russian.

## 5. Stream completions

By default, when you request a completion from the OpenAI, the entire completion is generated before being sent back in a single response.

If you're generating long completions, waiting for the response can take many seconds.

To get responses sooner, you can 'stream' the completion as it's being generated. This allows you to start printing or processing the beginning of the completion before the full completion is finished.

To stream completions, set `-Stream` switch as true.


### Downsides

Note that using stream completions, the only response you get is the response text generated, so you do not have access to any other details. For example, the number of tokens consumed by the request is no longer known, and it is no longer possible to maintain a history of past interactions using pipeline input.

In [ ]:
Request-ChatGPT 'Describe ChatGPT in 100 charactors.' -Stream | Write-Host -NoNewline

ChatGPT is an online community of knowledgeable strangers, connecting people through thoughtful conversations and insightful perspectives.

In [ ]:
# How much time is saved by streaming a chat completion

"1. Typical completions (no stream)"
$sw1 = [System.Diagnostics.Stopwatch]::StartNew()
$Response = Request-ChatGPT 'Describe ChatGPT in 100 words.' -Temperature 0
$sw1.Stop()
"Full response received {0} ms after request" -f $sw1.ElapsedMilliseconds

"`r`n-------------------------`r`n"

"2. Stream completions"
$counter = 0
$sw2 = [System.Diagnostics.Stopwatch]::StartNew()
Request-ChatGPT 'Describe ChatGPT in 100 words.' -Temperature 0 -Stream | % {
    if($counter -le 5){ "[{1}] Message chunk received {0} ms after request" -f $sw2.ElapsedMilliseconds, $counter }
    if($counter -eq 6){"..."}
    $counter++
}
$sw2.Stop()
"Full response received {0} ms after request" -f $sw2.ElapsedMilliseconds

1. Typical completions (no stream)


Full response received 3884 ms after request

-------------------------

2. Stream completions
[0] Message chunk received 292 ms after request
[1] Message chunk received 322 ms after request
[2] Message chunk received 353 ms after request
[3] Message chunk received 387 ms after request
[4] Message chunk received 419 ms after request
[5] Message chunk received 453 ms after request
...
Full response received 3743 ms after request


In a typical request without stream, it takes about 4 seconds to get an answer, but with stream, the time to finally get the full answer is not significantly different, but the first partial answer can be obtained in about 0.3 seconds.

### About output type of stream

Stream output is sequentially output to the PowerShell pipeline (standard output stream). Therefore, when the stream output is saved to a variable or displayed on the console, it is not a single string, but an array of small string chunks.


In [ ]:
$Response = Request-ChatGPT 'Describe ChatGPT in 100 charactors.' -Stream
"Output type: " + $Response.GetType().Fullname  # Array of strings
"Size of an array: " + $Response.Count
$Response[0..5]

# Array of string to single string
-join $Response

Output type: System.Object[]
Size of an array: 30
Chat
G
PT
 is
 a
 free
ChatGPT is a free online chat platform where you can talk to people from around the world on a variety of topics, and make new friends.


### How to display output to the console and save it to a variable as well

Stream output is simultaneously output to the information stream in addition to the standard output stream. This can be used to save to variables and display on the console at the same time.


In [ ]:
Request-ChatGPT 'Describe ChatGPT in 100 charactors.' -Stream -InformationVariable Response | Write-Host -NoNewline
""
-join $Response

ChatGPT is an online chat platform where people from around the world can connect and engage in conversations about various topics.
ChatGPT is an online chat platform where people from around the world can connect and engage in conversations about various topics.


## 6. Use with content moderation

This section describes how to test messages to / from ChatGPT for violations of OpenAI's content policy. `Request-Moderation` function provides this functionality.

### Simple senario

Before a message is entered to `Request-ChatGPT`, you can insert `Request-Moderation` into the pipeline. As `Request-Moderation` will output a warning message if the input text violates the content policy, specifying `-WarningAction` as `Stop` will stop processing before the message is passed on to the subsequent pipeline.

In [ ]:
'I want to kill them.' | Request-Moderation -WarningAction Stop | Request-ChatGPT

Write-Warning: Request-Moderation.ps1:102
Line |
 102 |  …             Write-Warning "This content may violate the content polic …
     |                ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The running command stopped because the preference variable "WarningPreference" or common
     | parameter is set to Stop: This content may violate the content policy. (violence)


### Complex senario

If you want to test not only input but also output messages of the AI, or if you want to perform custom processing when a policy is violated, you will need to be a bit more  complex procedure.

In [ ]:
$InputMessage = 'I want to kill them.'

# Test the input message.
$InputModeration = Request-Moderation -Text $InputMessage
if($InputModeration.results[0].flagged -eq $true){
    # Custom procedure whtn the input message violates the polocy.
    echo 'Input message is harmful.'
}
else{
    echo 'Input message is safe.'
}

# Get an answer from ChatGPT
$Response = Request-ChatGPT -Message $InputMessage -MaxTokens 15
echo $Response.Answer[0]

# Test the output message.
$OutputModeration = Request-Moderation -Text $Response.Answer[0]
if($OutputModeration.results[0].flagged -eq $true){
    # Custom procedure whtn the output message violates the polocy.
    echo 'Output message is harmful.'
}
else{
    echo 'Output message is safe.'
}

Input message is harmful.
I'm sorry, but as an AI language model, I can't cond
Output message is safe.


## 7. Miscellaneous options

`Request-ChatGPT` has some optional parameters.

+ `-Model`
+ `-Name`
+ `-RolePrompt`
+ `-Temperature`
+ `-TopP`
+ `-NumberOfAnswers`
+ `-StopSequence`
+ `-MaxTokens`
+ `-PresencePenalty`
+ `-FrequencyPenalty`
+ `-LogitBias`
+ `-User`
+ `-TimeoutSec`
+ `-MaxRetryCount`

### Make the response deterministic

If a small value such as `0` or `0.1` is specified for the `-Temperature` parameter, the response will be definitive. On the other hand, if a large value such as `0.8` is specified, the response will be random even if the same message is entered.

In [ ]:
$Message = 'Please output one female name, as appropriate.'
# Ask same question 5 times.
(1..5) | % {
    Request-ChatGPT -Message $Message -Temperature 0.1 | select -ExpandProperty Answer
}

Sophia.
Sophia
Sophia.
Sophia.
Sophia.


Since a small value was specified for temperature, the same question can be asked repeatedly and get almost same answers.

### Limit maximum length of output

Specifying the `-MaxTokens` parameter avoids unintentionally consuming too many tokens by generating excessively long responses.

In [ ]:
$Response = Request-ChatGPT -Message "Write a poem of about 200 words." -MaxTokens 20
$Response.Answer

The trees sway gently in the breeze
A symphony of rustling leaves
The sun sets over


A message that is expected to be quite long was used, but because a small value was specified for `-MaxTokens`, the output was terminated in the middle of the message.

### Eliminate specific words

`-LogitBias` can be used to increase or decrease the likelihood that a particular word (token) will be included in a response.

For example, let ChatGPT introduce itself without using the word "AI".

First, use the `ConvertTo-Token` command to find the token ID corresponding to the word "AI". It may be necessary to also find token IDs for some derived words to ensure that the targeted word is eliminated.


In [ ]:
$TargetWords = ('AI', ' AI', 'Ai', 'ai')
$TargetWords | ConvertTo-Token -Model 'gpt-3.5-turbo'

15836
15592
70767
2192


Next, a set of these tokens and values indicating the likelihood that these tokens will be used are created and specified in the `-LogitBias` parameter.

In this case, we specify a minimum value of `-100` to minimize the likelihood that the tokens will be used.

In [ ]:
$BiasMap = @{
    15836 = -100
    15592 = -100
    70767 = -100
    2192  = -100
}

$Response = Request-ChatGPT -Message 'Please introduce yourself in about 50 characters.' -LogitBias $BiasMap -Temperature 0
$Response.Answer

I am an artificial intelligence language model created by OpenAIGPT.


Though it reduced the likelihood of the word "AI" being included in the answer, but unfortunately, the word "OpenAIGPT" is included.

The word "OpenAIGPT" breaks down into four tokens:
+ `Open` : 5109
+ `A` : 32
+ `IG` : 1953
+ `PT` : 2898

Eliminating the token ID of 32 would eliminate the alphabet "A" and thus reduce the likelihood of getting a proper answer.

As such, manipulation by `-LogitBias` may not always work as expected.

Therefore, it may be better to simply devise a prompt and give instructions to the model to obtain an appropriate response.


In [ ]:
$Response = Request-ChatGPT `
    -Message 'Please introduce yourself in about 50 characters. But NEVER use the word "AI".' `
    -Temperature 0
$Response.Answer

I am an intelligent virtual assistant here to assist you.
